In [100]:
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup
import requests 
from nltk.corpus import stopwords 
import syllables
nltk.download('stopwords')
from nltk.stem import PorterStemmer
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pgemi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [101]:
#Open StopWords file
with open('StopWords_Generic.txt', 'r') as f:
    stop = f.read()
stop_words=list(stop)

#Open Master_dictionary
master=pd.read_excel('LoughranMcDonald_MasterDictionary_2018.xlsx')
master['Word']=master['Word'].str.lower()
master.set_index("Word", drop = False,inplace=True)

#Open uncertainty_dictionary
uncertain=pd.read_excel("uncertainty_dictionary.xlsx")
uncertain['Word']=uncertain['Word'].str.lower()

#Open constraining_dictionary
constraint=pd.read_excel("constraining_dictionary.xlsx")
constraint['Word']=constraint['Word'].str.lower()

#reading the cik_list file
df_raw=pd.read_excel("cik_list.xlsx")

In [102]:
#Declaring 30 List which will store all the values for text_files present in dataframe
#***********************SECTION-1 Variables************************
mda_Positive_Score=[]
mda_Negative_Score=[]
mda_Polarity_Score=[]
#***********************SECTION-2 Variables************************
mda_Avg_SenLen=[]
mda_Comp_Count=[]
mda_PComp_Count=[]
mda_Fog_Idx=[]
mda_Word_Count=[]
#*********************Section-3 Variables**************************
mda_Uncertain_Score=[]
mda_Constraint_Score=[]
#*********************Section-4 Variables**************************
mda_Positive_Score_W=[]
mda_Negative_Score_W=[]
mda_Uncertain_Score_W=[]
mda_Constraint_Score_W=[]


#***********************SECTION-1 Variables************************
qqdmr_Positive_Score=[]
qqdmr_Negative_Score=[]
qqdmr_Polarity_Score=[]
#***********************SECTION-2 Variables************************
qqdmr_Avg_SenLen=[]
qqdmr_Comp_Count=[]
qqdmr_PComp_Count=[]
qqdmr_Fog_Idx=[]
qqdmr_Word_Count=[]
#*********************Section-3 Variables**************************
qqdmr_Uncertain_Score=[]
qqdmr_Constraint_Score=[]

#*********************Section-4 Variables**************************
qqdmr_Positive_Score_W=[]
qqdmr_Negative_Score_W=[]
qqdmr_Uncertain_Score_W=[]
qqdmr_Constraint_Score_W=[]

#***********************SECTION-1 Variables************************
rf_Positive_Score=[]
rf_Negative_Score=[]
rf_Polarity_Score=[]
#***********************SECTION-2 Variables************************
rf_Avg_SenLen=[]
rf_Comp_Count=[]
rf_PComp_Count=[]
rf_Fog_Idx=[]
rf_Word_Count=[]
#*********************Section-3 Variables**************************
rf_Uncertain_Score=[]
rf_Constraint_Score=[]
#*********************Section-4 Variables**************************
rf_Positive_Score_W=[]
rf_Negative_Score_W=[]
rf_Uncertain_Score_W=[]
rf_Constraint_Score_W=[]
#*********************Section-5 Variables**************************
Constraint_Wh=[]

In [103]:
mda_reg = r"item[^a-zA-Z\n]*\d\s*\.\s*Management\'s Discussion and Analysis.*?^\s*item[^a-zA-Z\n]*\d\s*\.*"
qqdmr_reg = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Quantitative and Qualitative Disclosures about " \
            r"Market Risk.*?^\s*item\s*\d\s*"
rf_reg = r"item[^a-zA-Z\n]*\d[a-z]?\.?\s*Risk Factors.*?^\s*item\s*\d\s*"

In [104]:
stop_word = set(stopwords.words('english')) 
def tokenizer(token):
    tokens= [w.lower() for w in token if w.isalpha() and not w in stop_words and not w in stop_word]
    return tokens    

In [105]:
def scores(token):
    pos,ne,pol,un,co=0,0,0,0,0
    b=list(master['Word'])
    c=list(uncertain['Word'])
    d=list(constraint['Word'])
    for w in token:
        if w in b:
            pos+=master.loc[w,"Positive"]
            ne+=master.loc[w,"Negative"]
        if w in c:
            un+=1
        if w in d:
            co+=1
    pos=pos/2009
    ne=ne/2009
    pol+=(pos-ne)/(pos+ne+0.000001)
    return pos,ne,pol,un,co

In [106]:
def Average_SentenceLength(text,token):
    sen = text.split("\n") 
    sen = list(filter(None, sen))
    return len(token)/len(sen),len(sen)

In [107]:
def Complex_Count(token):
    #Stemming of each Word 
    porter = PorterStemmer()
    a=len(token)
    token=[porter.stem(w) for w in token]
    comp= [w for w in token if syllables.estimate(w)>2 ]
    return len(comp),len(comp)/a

In [108]:
def Constraint(token):
    d=list(constraint['Word'])
    co=0
    for w in token:
        if w in d:
            co+=1
    return co 

In [109]:
for index, row in df_raw.iterrows():
    url=row['SECFNAME']
    data =requests.get(url) 
    soup=BeautifulSoup(data.text, "lxml")
    mda=re.findall(mda_reg, soup.text, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    mda=(" ".join(mda)) 
    qqdmr=re.findall(qqdmr_reg, soup.text, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    qqdmr=(" ".join(qqdmr)) 
    rf=re.findall(rf_reg, soup.text, re.IGNORECASE | re.DOTALL | re.MULTILINE)
    rf=(" ".join(rf)) 
    #Calculate Constraining score for Whole Report
    tokens=tokenizer(soup.text.split(" "))
    Constraint_Wh.append(Constraint(tokens))    
    
    #Eliminate stop_words from file and nltk package and Eliminate punctuation
    if len(mda)==0:
        mda_Positive_Score.append(0),mda_Negative_Score.append(0),mda_Polarity_Score.append(0),mda_Avg_SenLen.append(0),
        mda_Comp_Count.append(0),mda_PComp_Count.append(0),mda_Fog_Idx.append(0),mda_Word_Count.append(0),
        mda_Uncertain_Score.append(0),mda_Constraint_Score.append(0),mda_Uncertain_Score_W.append(0),
        mda_Constraint_Score_W.append(0),mda_Positive_Score_W.append(0),mda_Negative_Score_W.append(0)
    if len(mda)>0:
        tokens= tokenizer(mda.split(" "))
        md_sen,sen=Average_SentenceLength(mda,tokens)
        md_comp,md_pcomp=Complex_Count(tokens)
        md_fog= 0.4*(len(tokens)/sen)+(md_comp/len(tokens))        
        md_pos,md_ne,md_pol,md_un,md_co=scores(tokens)
        
        mda_Positive_Score.append(md_pos),mda_Negative_Score.append(md_ne),mda_Polarity_Score.append(md_pol),mda_Avg_SenLen.append(md_sen),
        mda_Comp_Count.append(md_comp),mda_PComp_Count.append(md_pcomp),mda_Fog_Idx.append(md_fog),mda_Word_Count.append(len(tokens)),
        mda_Uncertain_Score.append(md_un),mda_Constraint_Score.append(md_co),mda_Uncertain_Score_W.append(md_un/len(tokens)),
        mda_Constraint_Score_W.append(md_co/len(tokens)),mda_Positive_Score_W.append(md_pos/len(tokens)),mda_Negative_Score_W.append(md_ne/len(tokens))
        
        
    if len(qqdmr)==0:
        qqdmr_Positive_Score.append(0),qqdmr_Negative_Score.append(0),qqdmr_Polarity_Score.append(0),qqdmr_Avg_SenLen.append(0),
        qqdmr_Comp_Count.append(0),qqdmr_PComp_Count.append(0),qqdmr_Fog_Idx.append(0),qqdmr_Word_Count.append(0),
        qqdmr_Uncertain_Score.append(0),qqdmr_Constraint_Score.append(0),qqdmr_Uncertain_Score_W.append(0),
        qqdmr_Constraint_Score_W.append(0),qqdmr_Positive_Score_W.append(0),qqdmr_Negative_Score_W.append(0)
    if len(qqdmr)>0:
        tokens= tokenizer(qqdmr.split(" "))
        qq_sen,sen=Average_SentenceLength(qqdmr,tokens)
        qq_comp,qq_pcomp=Complex_Count(tokens)
        qq_fog= 0.4*(len(tokens)/sen)+(qq_comp/len(tokens))        
        qq_pos,qq_ne,qq_pol,qq_un,qq_co=scores(tokens)
        
        qqdmr_Positive_Score.append(qq_pos),qqdmr_Negative_Score.append(qq_ne),qqdmr_Polarity_Score.append(qq_pol),qqdmr_Avg_SenLen.append(qq_sen),
        qqdmr_Comp_Count.append(qq_comp),qqdmr_PComp_Count.append(qq_pcomp),qqdmr_Fog_Idx.append(qq_fog),qqdmr_Word_Count.append(len(tokens)),
        qqdmr_Uncertain_Score.append(qq_un),qqdmr_Constraint_Score.append(qq_co),qqdmr_Uncertain_Score_W.append(qq_un/len(tokens)),
        qqdmr_Constraint_Score_W.append(qq_co/len(tokens)),qqdmr_Positive_Score_W.append(qq_pos/len(tokens)),qqdmr_Negative_Score_W.append(qq_ne/len(tokens))
    
    if len(rf)==0:
        rf_Positive_Score.append(0),rf_Negative_Score.append(0),rf_Polarity_Score.append(0),rf_Avg_SenLen.append(0),
        rf_Comp_Count.append(0),rf_PComp_Count.append(0),rf_Fog_Idx.append(0),rf_Word_Count.append(0),
        rf_Uncertain_Score.append(0),rf_Constraint_Score.append(0),rf_Uncertain_Score_W.append(0),
        rf_Constraint_Score_W.append(0),rf_Positive_Score_W.append(0),rf_Negative_Score_W.append(0)
    if len(rf)>0:
        tokens= tokenizer(rf.split(" "))
        rf_sen,sen=Average_SentenceLength(rf,tokens)
        rf_comp,rf_pcomp=Complex_Count(tokens)
        rf_fog= 0.4*(len(tokens)/sen)+(rf_comp/len(tokens))        
        rf_pos,rf_ne,rf_pol,rf_un,rf_co=scores(tokens)
        
        rf_Positive_Score.append(rf_pos),rf_Negative_Score.append(rf_ne),rf_Polarity_Score.append(rf_pol),rf_Avg_SenLen.append(rf_sen),
        rf_Comp_Count.append(rf_comp),rf_PComp_Count.append(rf_pcomp),rf_Fog_Idx.append(rf_fog),rf_Word_Count.append(len(tokens)),
        rf_Uncertain_Score.append(rf_un),rf_Constraint_Score.append(rf_co),rf_Uncertain_Score_W.append(rf_un/len(tokens)),
        rf_Constraint_Score_W.append(rf_co/len(tokens)),rf_Positive_Score_W.append(rf_pos/len(tokens)),rf_Negative_Score_W.append(rf_ne/len(tokens))

In [110]:
#Generate 43 columns for dataframe generated from cik_list 
df_raw["mda_positive_score"]=mda_Positive_Score
df_raw["mda_negative_score"]=mda_Negative_Score
df_raw["mda_polarity_score"]=mda_Polarity_Score
df_raw["mda_average_sentence_length"]=mda_Avg_SenLen
df_raw["mda_percentage_of_complex_words"]=mda_PComp_Count
df_raw["mda_fog_index"]=mda_Fog_Idx
df_raw["mda_complex_word_count"]=mda_Comp_Count
df_raw["mda_word_count"]=mda_Word_Count
df_raw["mda_uncertainty_score"]=mda_Uncertain_Score
df_raw["mda_constraining_score"]=mda_Constraint_Score
df_raw["mda_positive_word_proportion"]=mda_Positive_Score_W
df_raw["mda_negative_word_proportion"]=mda_Negative_Score_W
df_raw["mda_uncertainty_word_proportion"]=mda_Uncertain_Score_W
df_raw["mda_constraining_word_proportion"]=mda_Constraint_Score_W
df_raw["qqdmr_positive_score"]=qqdmr_Positive_Score
df_raw["qqdmr_negative_score"]=qqdmr_Negative_Score
df_raw["qqdmr_polarity_score"]=qqdmr_Polarity_Score
df_raw["qqdmr_average_sentence_length"]=qqdmr_Avg_SenLen
df_raw["qqdmr_percentage_of_complex_words"]=qqdmr_PComp_Count
df_raw["qqdmr_fog_index"]=qqdmr_Fog_Idx
df_raw["qqdmr_complex_word_count"]=qqdmr_Comp_Count
df_raw["qqdmr_word_count"]=qqdmr_Word_Count
df_raw["qqdmr_uncertainty_score"]=qqdmr_Uncertain_Score
df_raw["qqdmr_constraining_score"]=qqdmr_Constraint_Score
df_raw["qqdmr_positive_word_proportion"]=qqdmr_Positive_Score_W
df_raw["qqdmr_negative_word_proportion"]=qqdmr_Negative_Score_W
df_raw["qqdmr_uncertainty_word_proportion"]=qqdmr_Uncertain_Score_W
df_raw["qqdmr_constraining_word_proportion"]=qqdmr_Constraint_Score_W
df_raw["rf_positive_score"]=rf_Positive_Score
df_raw["rf_negative_score"]=rf_Negative_Score
df_raw["rf_polarity_score"]=rf_Polarity_Score
df_raw["rf_average_sentence_length"]=rf_Avg_SenLen
df_raw["rf_percentage_of_complex_words"]=rf_PComp_Count
df_raw["rf_fog_index"]=rf_Fog_Idx
df_raw["rf_complex_word_count"]=rf_Comp_Count
df_raw["rf_word_count"]=rf_Word_Count
df_raw["rf_uncertainty_score"]=rf_Uncertain_Score
df_raw["rf_constraining_score"]=rf_Constraint_Score
df_raw["rf_positive_word_proportion"]=rf_Positive_Score_W
df_raw["rf_negative_word_proportion"]=rf_Negative_Score_W
df_raw["rf_uncertainty_word_proportion"]=rf_Uncertain_Score_W
df_raw["rf_constraining_word_proportion"]=rf_Constraint_Score_W
df_raw["constraining_words_whole_report"]=Constraint_Wh

In [111]:
df_raw

,CIK,CONAME,FYRMO,FDATE,FORM,SECFNAME,mda_positive_score,mda_negative_score,mda_polarity_score,mda_average_sentence_length,...,rf_fog_index,rf_complex_word_count,rf_word_count,rf_uncertainty_score,rf_constraining_score,rf_positive_word_proportion,rf_negative_word_proportion,rf_uncertainty_word_proportion,rf_constraining_word_proportion,constraining_words_whole_report
0,3662,SUNBEAM CORP/FL/,199803,1998-03-06,10-K405,https://www.sec.gov/Archives/edgar/data/3662/0...,16.0,50.0,-0.515152,4.264789,...,0.000000,0,0,0,0,0.000000,0.000000,0.000000,0.000000,1115
1,3662,SUNBEAM CORP/FL/,199805,1998-05-15,10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,8.0,39.0,-0.659574,4.371324,...,0.000000,0,0,0,0,0.000000,0.000000,0.000000,0.000000,754
2,3662,SUNBEAM CORP/FL/,199808,1998-08-13,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.000000,0.000000,...,0.000000,0,0,0,0,0.000000,0.000000,0.000000,0.000000,4
3,3662,SUNBEAM CORP/FL/,199811,1998-11-12,10-K/A,https://www.sec.gov/Archives/edgar/data/3662/0...,34.0,102.0,-0.500000,3.985896,...,0.000000,0,0,0,0,0.000000,0.000000,0.000000,0.000000,529
4,3662,SUNBEAM CORP/FL/,199811,1998-11-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/3662/0...,0.0,0.0,0.000000,0.000000,...,0.000000,0,0,0,0,0.000000,0.000000,0.000000,0.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,12239,SPHERIX INC,200704,2007-04-02,10-K,https://www.sec.gov/Archives/edgar/data/12239/...,0.0,0.0,0.000000,0.000000,...,0.000000,0,0,0,0,0.000000,0.000000,0.000000,0.000000,101
148,12239,SPHERIX INC,200705,2007-05-16,NT 10-Q,https://www.sec.gov/Archives/edgar/data/12239/...,0.0,0.0,0.000000,0.000000,...,0.000000,0,0,0,0,0.000000,0.000000,0.000000,0.000000,2
149,12239,SPHERIX INC,200705,2007-05-18,10-Q,https://www.sec.gov/Archives/edgar/data/12239/...,0.0,0.0,0.000000,0.000000,...,0.871062,5,39,8,0,0.000000,0.025641,0.205128,0.000000,16
150,12239,SPHERIX INC,200705,2007-05-23,10-K/A,https://www.sec.gov/Archives/edgar/data/12239/...,0.0,0.0,0.000000,0.000000,...,1.675553,135,917,48,8,0.022901,0.051254,0.052345,0.008724,104


In [113]:
df_raw.to_excel("Output_Data_Structure.xlsx")  